## Applied Data Science Capstone Week 3 Assignment Part 1 of 3 

### Importing all needed libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# import packages for web scrapping: beautifulsoup & requests
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen

print('Libraries imported.')


Libraries imported.


### Scraping of the Wikipedia page

In [20]:
# Get the needed postal code data for Canada from the Wikipedia page by using Beautifulsoup method
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = urlopen(url)
html = BeautifulSoup(html) 
html
print(html.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"3a9e02e6-6edb-405c-9f8a-c28ce3880736","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":968481359,"wgRevisionId":968481359,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toron

In [21]:
# Get the title, text and corresponding headers; get column headers
html.title
html.title.text
wikitable = html.find_all('table', class_="wikitable sortable") 
headers = wikitable[0].find_all('th')
column = [i.text.strip() for i in headers]
data = wikitable[0].find_all('td') 
total_cell, total_row = len(data), len(data)/3


### Creating the dataframe with three columns: PostalCode, Borough, and Neighborhood

In [22]:
# Create the lists for the three columns
PostalCode = [] 
for i in range(0, total_cell, 3):    
    k = data[i].text.strip() 
    PostalCode.append(k)
    
Borough = [] 
for i in range(1, total_cell, 3):    
    k = data[i].text.strip() 
    Borough.append(k) 

Neighborhood = [] # create list named "col_3"
for i in range(2, total_cell, 3):    
    k = data[i].text.strip() 
    Neighborhood.append(k) 

In [23]:
# Add the lists into a ditionary and build the dataframe
Canada_df = {}
for a, b in zip(column, [PostalCode, Borough, Neighborhood]): 
    Canada_df[a] = b 
    
Canada_postal_data = pd.DataFrame(Canada_df) 
Canada_postal_data.head() 

,Post Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [25]:
Canada_postal_data.rename(columns = {'Post Code':'PostalCode'}, inplace = True)
Canada_postal_data.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Remove the not assigned Boroughs and reset index

In [26]:
# Removal of the "Not assigned" values in "Borough"
Canada_df_cleaned = Canada_postal_data.query("Borough != 'Not assigned'")
Canada_df_cleaned

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [27]:
Canada_df_cleaned_2  = Canada_df_cleaned.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
Canada_df_cleaned_2.columns = ['PostalCode', 'Borough', 'Neighborhood']
Canada_df_cleaned_2

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


### Print number of rows using Shape Method

In [28]:
Canada_df_cleaned_2.shape

(103, 3)

## Applied Data Science Capstone Week 3 Assignment Part 2 of 3

### Importing all needed libraries

In [18]:
conda install --channel conda-forge geopandas

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [29]:
# Get Geocoding libraries
import geopy
import geopandas
print('Libraries imported.')


Libraries imported.


### Use libraries and csv file provided in the task to get longitude and latitude values

In [30]:
#Using the libraries to find longitude & latitude values
Canada_geo = pd.read_csv('http://cocl.us/Geospatial_data')
Canada_geo.columns = ['PostalCode', 'Latitude', 'Longitude']

### Merging the dataframes

In [31]:
#Create the new data frame by merging the dataframe from part 1 with the dataframe containing the geo data
Canada_merged = pd.merge(Canada_df_cleaned_2, Canada_geo, on=['PostalCode'], how='inner')
Canada_combined = Canada_merged[['PostalCode','Borough', 'Neighborhood',  'Latitude', 'Longitude']].copy()
Canada_combined.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Applied Data Science Capstone Week 3 Assignment Part 3 of 3


### Import the necessary libraries

In [32]:
# Import libraries for exlporing Toronto maps and do clustering
from pandas.io.json import json_normalize 
import json
from sklearn.cluster import KMeans
print('Libraries imported.')

Libraries imported.


### Exploring Toronto in Maps and Clustering

In [38]:
Canada_Toronto = Canada_combined.copy()
Canada_Toronto = Canada_Toronto[Canada_combined.Borough.str.contains("Toronto")]
Canada_Toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [43]:
Canada_Toronto_Cluster_Map = folium.Map(location=[43.65, -79.4], zoom_start=12)

X = Canada_Toronto['Latitude']
Y = Canada_Toronto['Longitude']
Z = np.stack((X, Y), axis=1)

kmeans = KMeans(n_clusters=4, random_state=0).fit(Z)

clusters = kmeans.labels_
colors = ['red', 'green', 'blue', 'yellow']
Canada_Toronto['Cluster'] = clusters

for latitude, longitude, borough, cluster in zip(Canada_Toronto['Latitude'], Canada_Toronto['Longitude'], Canada_Toronto['Borough'], Canada_Toronto['Cluster']):
    label = folium.Popup(borough, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color=colors[cluster],
        fill_opacity=0.7).add_to(toronto_map)  

Canada_Toronto_Cluster_Map

In [33]:
# Retreive coordinates for Toronto
address = 'Toronto, Ontario'
geolocator = geopy.Nominatim(user_agent="ny_explorer", timeout=30)
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [37]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map based on coordinates of postcode
for lat, lng, PostalCode in zip(Canada_geo['Latitude'], Canada_geo['Longitude'], Canada_geo['PostalCode']):
    label = '{}'.format(PostalCode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)    
map_toronto